This file is part of GaPSE
Copyright (C) 2022 Matteo Foglieni

GaPSE is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

GaPSE is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
General Public License for more details.

You should have received a copy of the GNU General Public License
along with GaPSE. If not, see <http://www.gnu.org/licenses/>.
```


```

# GaPSE TUTORIAL

This is a small tour of the Galaxy Power Spectrum Estimator software. It shows
how the program can be easily used inside a Jupyter Notebook.

However, if you prefer, you can also write the commands you are interested to execute inside the file "GaPSE-exe.jl", and then run in the command line

```
$ julia GaPSE-exe.jl
```

Furthermore, almost all the functions implemented inside GaPSE have a detailed documentation; feel free to check it whenever you want with the usual Julia command `?NAME` (where `NAME` is the name of the function/object you are interested in).

For this tutorial we will show the computation of the Two-Point Correlation Functions (TPCFs) concerning the relativistic Galaxy Number Counts (GNC) perturbations. The code may computes in a very intuitive way also the Luminosity Distance (LD) one's and the cross correlations GNCxLD and LDxGNC.

## Path to GaPSE

First of all, we have to include GaPSE in our Jupyter Notebook:

In [1]:
# In case you prefer to use GaPSE in a Jupyter Notebook in a "distant" directory,
# you only have to set this path equal to the directory GaPSE is stored in.
PATH_TO_GAPSE = "../"; 

In [2]:
include(PATH_TO_GAPSE * "src/GaPSE.jl");
# NOTE : do not run this cell multiple times! In you have a problem, restart
# the kernel an re-run all the cells.

In [3]:
using DelimitedFiles, Plots, Printf, LaTeXStrings 
# These are useful libraries for plots and reading files, but they are not
# mandatory in order to run the code

```




```

## Creation of a Cosmology 

The fundamental struct of GaPSE is `Cosmology`; it contains all the 
relevant information for future computations.

Let's have a look at the `Cosmology` constructor:

```julia
Cosmology(params::CosmoParams, file_data::String,
    file_ips::String, file_windowF::String,
    file_IntwindowF::String)
```


As we can see, in order to create such a struct, we must give a `params::CosmoParams` struct (we will see it later) and 4 input files:
- `file_data` : the background data for the Universe considered;
- `file_ips` : the matter power spectrum at redshift z=0 we are focused on;
- `file_windowF` : a map on μ and x of the window function F (use the default one initially, provided inside GaPSE directory `data/`);
- `file_IntwindowF` : a map on μ and s of the window function F integrated over `s_1` (again, use the default one initially, provided inside GaPSE directory `data/`);
  
The first two files should be produced by the [CLASS](https://github.com/lesgourg/class_public) software. As an example, we take the two stored inside `data/`:

In [ ]:
FILE_PS = PATH_TO_GAPSE * "data/WideA_ZA_pk.dat";
FILE_BACKGROUND = PATH_TO_GAPSE * "data/WideA_ZA_background.dat";
FILE_F_MAP = PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt";
FILE_IF_MAP = PATH_TO_GAPSE * "data/IntegrF_REFERENCE_pi2_z005020.txt";

Now we have to set the dimensions of the survey we are considering. 

GaPSE assumes that the survey has an azimutally simmetric volume, contained inside two concentric spheres (with radius specified by the input minimum and maximum redshifts `z_min` and `z_max`) and with angular extensions `2 θ_max` (so `θ_max` is the angle between the central axis and the edge). The extreme redshifts are internally translated into extreme comoving distances $s_\mathrm{min}$ and $s_\mathrm{max}$.

Here we show a orthogonal projection on the plane containing the three points $O$ (the observer), $\mathbf{s}_\mathrm{min}$ and $\mathbf{s}_\mathrm{max}$.

<img src="images/survey.png" style="width: 400px;">

We also show the comoving effective distance $s_\mathrm{eff}$, which is the counterpart of the effective redshift $z_\mathrm{eff}$ calculated as follows:

$$
\begin{split}
z_\mathrm{eff} := 
    \frac{
        \int \mathrm{d}^3\mathbf{s} \, \phi^2(\mathbf{s}) \, z(s)
     }{
         \int \mathrm{d}^3\mathbf{s}\, \phi^2(\mathbf{s}) 
      } &= \frac{
          \int_0^\infty \mathrm{d}s  \, s^2 \, \phi^2(s) \, z(s) \times
          \int_{4\pi}\mathrm{d}^2\hat{\mathbf{s}} \, W^2(\hat{\mathbf{s}})
      }{
          \int_0^\infty \mathrm{d}s \, s^2 \, \phi^2(s)\times
          \int_{4\pi}\mathrm{d}^2\hat{\mathbf{s}} \, W^2(\hat{\mathbf{s}})
      } \\[5pt]
      &= \frac{
          \int_0^\infty \mathrm{d}s  \, s^2 \, \phi^2(s) \, z(s)
      }{
          \int_0^\infty \mathrm{d}s \, s^2 \, \phi^2(s)
      } \\[4pt]
      &= \frac{3}{s_\mathrm{max}^3 - s_\mathrm{min}^3} \,
          \int_{s_\mathrm{min}}^{s_\mathrm{max}} \mathrm{d}s  \, s^2 \, z(s)
\end{split}
$$

where $\phi(s)$ and $W(\hat{s})$ are respectivelly the radial and angulal window functions; it is assumed that the survey window function $\phi(\mathbf{s})$ can be separated into a radial and angular part:

$$
\phi(\mathbf{s}) = \phi(s) \, W(\hat{s})
$$

$$ \phi(s, s_\mathrm{min}, s_\mathrm{max}) = 
    \begin{cases} 1 \quad s_\mathrm{min} < s < s_\mathrm{max} \\ 0 \quad \mathrm{otherwise}\end{cases} $$
    
 $$ W(\hat{s}, \theta_\mathrm{MAX}) = 
    \begin{cases} 1 \quad 0 \leq \theta < \theta_\mathrm{MAX} \\ 0 \quad \mathrm{otherwise}\end{cases} $$

If you are not interested in a specific survey, but only in the "true" TPCFs of the Universe considered, do not worry: let the default values for these fields, in the functions we are going to show you will only have 
to set `use_window = false`.

For this demo case, let's set the following survey volume values:

In [ ]:
z_min, z_max, θ_max = 0.05, 0.20, π / 2.0;

At this point, we have to create the `CosmoParams` struct previously mentioned, that will contains all the options to be passed to `Cosmology`.
Beyond the already mentioned `z_min`, `z_max` and `θ_max` (which are mandatory), you may choose to specify:
- `Ω_b::Float64` and `Ω_cdm::Float64` : barionic and cold-dark-matter density parameters

- `h_0::Float64` : today's Hubble adimensional parameter ($H_0 = h_0 \times 100 \; \mathrm{km}/(\mathrm{s}  \, \mathrm{Mpc})$).

- `b::Float64` : galaxy bias.

- `s_b::Float64` : magnification bias, i.e. the slope of the luminosity function at the luminosity threshold.

- `𝑓_evo::Float64` : evolution bias.

- many others; if you are interested, check the documentation of `CosmoParams`.

Here we provided the full list of them, with all their default values.

In [ ]:
params = GaPSE.CosmoParams(z_min, z_max, θ_max;
    Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70, s_lim=1e-2,
    s_b = 0.0, 𝑓_evo = 0.0, b = 1.0,
    IPS_opts = Dict(
        :fit_left_min => 1e-6, :fit_left_max => 3e-6,
        :fit_right_min => 1e1, :fit_right_max => 2e1),
    IPSTools_opts = Dict(
        :N => 1024, :fit_min => 0.05, :fit_max => 0.5, 
        :con => true, :k_min => 1e-8, :k_max => 10.0),
    WFI_opts = Dict(
        :llim => 0.0, :rlim => Inf, :N => 1000,
         :trap => true, :rtol => 1e-2, :atol => 0.0,
         :ss_start => 0.0, :ss_step => 10.0,
         :ss_stop => 0.0),
);

Finally, we can create the `Cosmology` (let's call it `cosmo`) with all the inputs specified before!
We also print on the screen all the options stored by that `Cosmology` itself, in order to be sure that everything is okay, through the function `parameters_used`.

In [ ]:
cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, 
    FILE_F_MAP, FILE_IF_MAP);

GaPSE.parameters_used(stdout, cosmo);

Now we are ready to calculate the TPCF and/or PS associated with each of the GR effects for this Universe!

```




```

## TPCF of a single effect

The Genearal Relativistic (GR) effects affecting GNC, following the conventions used in Castorina, Di Dio (2020), are 5: Standard Newtonian, Doppler, Lensing, Local GP and Integrated GP (where GP stands for Gravitational Potential). See th

Consequently, the TPCFs associated are $5\times5=25$ (5 are auto-correlations, and 20 the cross-ones; note that they are not simmetric, so for instance Lensing-Doppler is different from Doppler-Lensing).

Their names are, with obvious associations, the ones contained inside the
`GaPSE.GR_EFFECTS_GNC` vector:

In [ ]:
GaPSE.GR_EFFECTS_GNC

As a first example, we will focus on the Doppler auto-correlation function, identified by the name `auto_doppler`.

How we may calculate the value of such TPCF, for the considered Cosmology `cosmo`?
GaPSE provides three functions:

- `ξ_GNC_multipole`
- `map_ξ_GNC_multipole`
- `print_map_ξ_GNC_multipole`

The frame of reference you have to keep in mind is the following:

<table><tr>
<td> <img src="images/s1s2.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="images/s1s2_inside_survey.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

GaPSE assumes (unless differently specified) that $\mathbf{s}_1$ is on the $s_\mathrm{eff}$ sphere (and so $s_1 = s_\mathrm{eff}$). The second input $s$ is the comoving distance from $s_1$, as you can see from the sketch.

These are the transformations concerning the two coordinate systems:

$$
y = \cos \theta = \hat{\mathbf{s}}_1 \cdot  \hat{\mathbf{s}}_2 
\quad , \quad
\mu = \cos \phi = \hat{\mathbf{s}}_1 \cdot  \hat{\mathbf{s}}
$$

$$
s(s_1, s_2, y) = \sqrt{s_1^2 + s_2^2 - 2 \, s_1 \, s_2 \, y}
\quad , \quad
s_2(s_1, s, \mu) = \sqrt{s_1^2 + s^2 + 2 \, s_1 \, s \, \mu}
$$

$$
\mu(s_1, s_2, y) = \frac{y \, s_2 - s_1}{s(s_1, s_2, y)}
\quad , \quad
y(s_1, s, \mu) = \frac{\mu \, s + s_1}{s_2(s_1, s, \mu)} 
$$

```


```

### ξ_GNC_multipole

```julia
ξ_GNC_multipole(s1, s, effect::String, 
    cosmo::Cosmology; 
    L::Integer = 0, use_windows::Bool = true,
    alg::Symbol = :lobatto,
    N_lob::Int = 100, N_trap::Int = 100, 
    atol_quad::Float64 = 0.0, rtol_quad::Float64 = 1e-2, 
    kwargs...) ::Float64
``` 

Returns the value of the chosen GNC GR `effect` multipole (of order `L`) calculated in `s1` and a distance `s` from it, in the given Cosmology `cosmo`.

As previously explained, you have to set the flag `use_windows = false` if you are not interested in a specific survey, and `use_windows = true` (or not set the flag at all, since `true` is the default value) if you are.

The computation of the integral through $\mu$ might be performed using one between the 3 following algorithm. In order to choose which to use, set th keyword `alg` to the corresponding symbol:
- `alg = :quad` : Adaptive Gauss-Kronrod quadrature, using `quadgk` of the native Julia package [QuadGK.jl](https://github.com/JuliaMath/QuadGK.jl). It works perfectly for the monopoles, but does not always converge for higher order multipoles. Specify its relative and absolute (FLOATING POINT) tolerance through the keywords `atol_quad` and `rtol_quad` (the default values `0.0` and `1e-2` should work always fine).
- `alg = :lobatto` : Fast Gaussian-Lobatto quadrature, through `gausslobatto` of the Julia package [FastGaussQuadrature.jl](https://github.com/JuliaApproximation/FastGaussQuadrature.jl); you can specify the (INTEGER) number of points to be used through the keyword `N_lob` (default is 100, but we recommed to use even 1000 for a more precise calculus). It works good everywhere, but it's slower than `quad` (so we recommend to use it only for L=1,2,3,4 and to use `quad` for L=0).
- `alg = :trap` : basic Trapezoidal rule, through `trapz` of the Julia package [Trapz.jl](https://juliapackages.com/p/trapz); you can specify the (INTEGER) number of points to be used through the keyword `N_trap` (default is 100, but we recommed to use even 1000 for a more precise calculus). HOWEVER, HTIS IS BY FAR THE WORSE METHOD, AVOID TO USE IT IF NOT FOR SPECIFIC REASONS!

Let's calculate the value of the `auto_doppler` effect CF in `s1 = s_eff` and `s = 50` (all the distances are measured in $h_0^{-1} \, \mathrm{Mpc}$), both with and without window function. You may try to change the `s` value to another one you prefer.

NOTE: GaPSE is conceived for computation on large scales; at the moment, the values for small comoving distance `s` (under few $h_0^{-1}\,\mathrm{Mpc}$) are not correct, especially for high order multipoles (`L=1,2,...`)

In [ ]:
GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, "auto_doppler", cosmo; 
    L = 0, use_windows = false, alg = :quad)

In [ ]:
GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, "auto_doppler", cosmo; 
    L = 0, use_windows = true, alg = :quad)

In [ ]:
GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, "auto_doppler", cosmo; 
    L = 0, alg = :quad)

In [ ]:
q = GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, "auto_doppler", cosmo; L = 0, alg = :quad)
l = GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, "auto_doppler", cosmo; L = 0, alg = :lobatto, N_lob = 1000)
t = GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, "auto_doppler", cosmo; L = 0, alg = :trap, N_trap = 1000)
q,t,l

NOTE: it would appear natural, in the Julia language, to use the broadcast operator and/or comprehension lists in order to calculate the chosen GR `effect` multipole in `s1` (fixed) for a vector of `s` distances from it `ss`, in the given Cosmology `cosmo`. In other words, it may seem straightforward to write:

```julia
    ss = range(1, 1000, length = 100)
    res = [GaPSE.ξ_multipole(cosmo.s_eff, s, 
        "auto_doppler", cosmo; L = 0) 
        for s in ss]
```

Do not do it: due to the way the program is build, it is computationally faster
and more accurate to use other GaPSE functions (`map_ξ_GNC_multipole` and so on, which we will present in the next section).

And for the next multipoles? Simply set the keyword parameter `L` to the order you need.

In [ ]:
vec = [GaPSE.ξ_GNC_multipole(cosmo.s_eff, 50, 
        "auto_doppler", cosmo; L = L, alg = :lobatto, use_windows = false) 
        for L in 0:4]
for (i, r) in enumerate(vec)
    println("L = $(i-1) :  xi^{Doppler} = $r")
end

```

```

### map_ξ_GNC_multipole

```julia
map_ξ_GNC_multipole(cosmo::Cosmology, effect::String,
    ss::Union{Vector{Float64},Nothing} = nothing; 
    s1 = nothing, kwargs... 
    ) ::Tuple{Vector{Float64}, Vector{Float64}}
```

For an input vector of distances `ss`, return a tuple with
- `ss` itself as first element
- the vector of values of the chosen GR `effect` multipole (of order `L`) calculated in `s1` for all the distances `s` contained in the vector `ss` (in the given Cosmology `cosmo`) as second element

If, as by default, `ss = nothing`, it is interally set `ss = 10 .^ range(0, log10(2.0*cosmo.s_max), length=N_log)`, where `N_log` is a keyword argument; this is why it is returned `ss`.

Likewise, if `s1 = nothing` (as by default), it is internally set `s1 = cosmo.s_eff`.
The keywords are shared with `ξ_GNC_multipole`, so you can set/unset the window, select algorithm, choose monopole and so on asa before.


IMPORTANT: USE LOGARITHMICALLY DISTRUBUTED POINTS FOR `ss`!
It's really important not for the `map_ξ_GNC_multipole`function or its family, but for the Power Spectrum computation algorithm we'll see later.

For instance, we might calculate the `auto_doppler` effect monopole CF in a linearly spaced set of points `range(1, 1000, length=300)`, both with and without window functions:

In [ ]:
XS = range(1, 1000, length = 300)
xs1, ys1 = GaPSE.map_ξ_GNC_multipole(cosmo, "auto_doppler", XS; 
    L = 0, use_windows = false, alg = :quad);
xs2, ys2 = GaPSE.map_ξ_GNC_multipole(cosmo, "auto_doppler", XS; 
     L = 0, use_windows = true, alg = :quad);


pyplot()
plot(xs1, ys1 .* xs1 .^2,
    #xaxis=:log, #yaxis=:log, 
    lc=:red, ls=:solid, size=(500, 300),
    title="auto_doppler TPCF", label = "no F",
    ylabel=L"s^2 \; \xi^{v_{\parallel} \, v_{\parallel}}_{L=0}(s)",
    xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
    legend = :topright,
)
plot!(xs2, ys2 .* xs2 .^2,
    #xaxis=:log, #yaxis=:log, 
    lc=:blue, ls=:solid, size=(500, 300), label = "with F",
    ylabel=L"s^2 \; \xi^{v_{\parallel} \, v_{\parallel}}_{L=0}(s)",
    xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
)

What if instead we are not interested in a survey and so the default `s1 = s_eff` is useless? Simply we set `s1` to the value we are interested in! Supposing `s1 = 400`, let's compare this one with the previous one without windows: 

In [ ]:
XS = range(1, 1000, length = 300)
xs1, ys1 = GaPSE.map_ξ_GNC_multipole(cosmo, "auto_doppler", XS; 
    L = 0, use_windows = false, alg = :quad)
xs2, ys2 = GaPSE.map_ξ_GNC_multipole(cosmo, "auto_doppler", XS; 
    s1 = 400, L = 0, use_windows = false, alg = :quad)


pyplot()
plot(xs1, ys1 .* xs1 .^2,
    #xaxis=:log, #yaxis=:log, 
    lc=:red, ls=:solid, size=(500, 300),
    title="auto_doppler TPCF", label = "s1 = s_eff",
    ylabel=L"s^2 \; \xi^{v_{\parallel} \, v_{\parallel}}_{L=0}(s)",
    xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
    legend = :topright,
)
plot!(xs2, ys2 .* xs2 .^2,
    #xaxis=:log, #yaxis=:log, 
    lc=:blue, ls=:solid, size=(500, 300), label = "s1 = 400",
    ylabel=L"s^2 \; \xi^{v_{\parallel} \, v_{\parallel}}_{L=0}(s)",
    xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
)

```



```

### print_map_ξ_GNC_multipole

```julia
print_map_ξ_GNC_multipole(cosmo::Cosmology, out::String, 
    effect::Union{String,Function}, ss = nothing;
    s1 = nothing, kwargs...)
```

Same as `map_ξ_GNC_multipole`, but the two resulting vectors are printed in the file `out`, together with all the options used for their computations.

For instance, let's print on a file named `my_first_doppler.txt` the `auto_doppler`TPCF monopole, calculated in the vector of values `ss = 10 .^ range(0, 3, length = 1000)`.

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(cosmo, "my_first_doppler.txt", 
    "auto_doppler", 10 .^ range(0, 3, length = 1000); 
    L = 0, use_windows = false, alg = :quad);

Here we plot the result:

In [ ]:
xs, ys = GaPSE.readxy("my_first_doppler.txt", comments=true)

pyplot()
plot(xs, ys .* xs .^2,
    #xaxis=:log, #yaxis=:log, 
    lc=:red, ls=:solid,size=(500, 300),
    title="auto_doppler TPCF",
    ylabel=L"s^2 \; \xi^{v_{\parallel} \, v_{\parallel}}_{L=0}(s)",
    xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
    legend=false,
)

```


```

## TPCF of all the effects

The most straightforward path to calculate all the CFs (and also their sum) is through the three following functions:

- `sum_ξ_GNC_multipole`
- `map_sum_ξ_GNC_multipole`
- `print_map_sum_ξ_GNC_multipole`

They work in the exactly same way of their corresponding "single-effect" one. The only difference is that the output is now not a single GR effect, but all of them plus their sum.

```

```

### sum_ξ_GNC_multipole

```julia 
sum_ξ_GNC_multipole(s1, s, cosmo::Cosmology; kwargs...
    )::Tuple{Float64,Vector{Float64}}
```
For input distances `s1` ans `s` and in the given Cosmology `cosmo`, returns a tuple which has:
- the sum of all the 16 GNC TPCFs multipoles (of order `L`) calculated in `s1` and a distance `s` from it, in the given Cosmology `cosmo`.
- a vector of `Float64` values containing each off the 16 TPCF terms; they are ordered following the `GaPSE.GR_EFFECTS_GNC` one

In [ ]:
s = 1000
res_sum, res_xis = GaPSE.sum_ξ_GNC_multipole(cosmo.s_eff, s, 
    cosmo; use_windows = false, L = 0, alg = :quad);
res_sum, res_xis

In [ ]:
names = [
    L"\delta \, \delta", 
    L"v_{\parallel} \, v_{\parallel}", 
    L"\kappa \, \kappa", 
    L"\phi \, \phi", 
    L"\int\phi \, \int\phi", 
    
    
    L"\delta \, v_{\parallel}", L"v_{\parallel} \, \delta",
    L"\delta \, \kappa", L"\kappa \, \delta",
    L"\delta \, \phi", L"\phi \, \delta",
    L"\delta \, \int\phi", L"\int\phi \, \delta",
    
    L"\kappa \, v_{\parallel}", L"v_{\parallel} \, \kappa",
    L"v_{\parallel} \, \phi", L"\phi \, v_{\parallel}",
    L"v_{\parallel} \, \int\phi", L"\int\phi \, v_{\parallel}",
    
    L"\kappa \, \phi", L"\phi \, \kappa",
    L"\kappa \, \int\phi", L"\int\phi \, \kappa",
    
    L"\phi \, \int\phi", L"\int\phi \, \phi"
];
leg = L"\delta"*" = Newton    \n"*L"v_{\parallel}"*" = Doppler    \n" *
        "κ = Lensing  \n"*"ϕ = Local GP  \n"*"∫ϕ = Integrated GP"

pyplot()
ys = [res_xis[i] for i in 1:length(res_xis)];
plot(1:25, ys .* (s^2), seriestype=:scatter, 
    series_annotations = [text(name; pointsize=7, valign=:bottom) 
        for name in names], 
    mc=vcat([:red for i in 1:5], [:blue for i in 6:25]), 
    markerstrokewidth = 0, legend=false,
    xlabel = "ξ considered",  xticks=false, size = (800, 400),
    ylabel = L"s^2 \, \xi^{i}_{L=0}(s)," * 
    L" \quad \mathrm{with} \; s = %$s \; h_0^{-1}\mathrm{Mpc}",
    ylims = (-1.10, 0.61), yticks = -1.0:0.25:0.5,
    title = "Contribution at s = $s Mpc/h_0 of each GNC ξ"
    
)

annotate!([(12, -0.6, text(leg; pointsize=7))])
#annotate!([(12, 0.8*max(ys.* (s^2)...), text(leg; pointsize=7))])

```


```

### map_sum_ξ_GNC_multipole

```julia
map_sum_ξ_GNC_multipole(cosmo::Cosmology, 
    ss = nothing; L::Integer = 0, s1 = nothing, 
    kwargs...
    )::Tuple{Vector{Float64}, Vector{Float64},
        Vector{Vector{Float64}}}
```

As for `map_ξ_multipole`, returns also the vector `ss`. It's now prettly useless to spend more words on this instead of simply see it in action, isn't it?

In [ ]:
ss, res_sums, res_xis = GaPSE.map_sum_ξ_GNC_multipole(cosmo, 
    10 .^ range(0, 3, length = 100), use_windows = false, L = 0, alg = :quad);

In [ ]:
func_ξs_names = [
    L::Int -> L"$\xi^{\delta \, \delta}_{L=%$L}(s)$", 
    L::Int -> L"$\xi^{v_{\parallel} \, v_{\parallel}}_{L=%$L}(s)$", 
    L::Int -> L"$\xi^{\kappa \, \kappa}_{L=%$L}(s)$", 
    L::Int -> L"$\xi^{\phi \, \phi}_{L=%$L}(s)$", 
    L::Int -> L"$\xi^{\int\phi \, \int\phi}_{L=%$L}(s)$", 
    
    L::Int -> L"$\xi^{\delta \, v_{\parallel}}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{v_{\parallel} \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\delta \, \kappa}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\kappa \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\delta \, \phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\phi \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\delta \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\int\phi \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\kappa \, v_{\parallel}}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{v_{\parallel} \, \kappa}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{v_{\parallel} \, \phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\phi \, v_{\parallel}}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{v_{\parallel} \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\int\phi \, v_{\parallel}}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\kappa \, \phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\phi \, \kappa}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\kappa \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\int\phi \, \kappa}_{L=%$L}(s)$",
    
    L::Int -> L"$\xi^{\phi \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$\xi^{\int\phi \, \phi}_{L=%$L}(s)$",
];

In [ ]:
pyplot()
ps = [plot() for i in 1:length(GaPSE.GR_EFFECTS_GNC) + 1]

title = plot(title = "GNC Correlation functions terms", grid=false, 
    showaxis = false, bottom_margin = -50Plots.px, ticks=false)

legend_pos=[
    :bottomleft,
    :bottomleft, :topright, :bottomleft, :bottomleft,
    :topleft, :topleft, :topleft, :topleft,
    :bottomleft, :bottomleft, :bottomleft, :bottomleft,
    :bottomleft, :bottomleft, :bottomleft, :bottomleft,
]

xs = ss

ps[1] = plot(xs, res_sums .* xs .^2, size=(300, 100),
        lw=2, lc="red", ls=:solid,
        title="All summed xis", 
        xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
        ylabel=L"s^2 \, \xi^{\mathrm{ALL}}_{L=0}(s)",
        #xaxis=:log, #yaxis=:log,
        top_margins=15Plots.px, legend=false)

for i in 1:length(GaPSE.GR_EFFECTS_GNC)
    ys = res_xis[i]

    ps[i+1] = plot(
        xs, ys .* xs .^2, 
        size=(300, 100),
        lw=2, lc="red", ls=:solid,
        title=GaPSE.GR_EFFECTS_GNC[i], 
        xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
        ylabel=L"s^2 \, " * func_ξs_names[i](0),
        #xaxis=:log, #yaxis=:log,
        top_margins=15Plots.px,
        legend = false,
    )
end

l = @layout [A{0.02h}; grid(2,3){0.22h} ; grid(5,4){0.75h} ]
P = plot(title, ps..., layout=l, 
    left_margins = 15Plots.px, right_margins = 30Plots.px,
    size=(1200,1800))

```



```

### print_map_sum_ξ_GNC_multipole

```julia 
print_map_sum_ξ_GNC_multipole(cosmo::Cosmology, 
    out::String, ss = nothing; 
    single::Bool = true, 
    kwargs...)
```
As `print_map_ξ_GNC_multipole`, prints the output in the file `out`.

The only new thing to point out is the bool `single`: if `true`, all the CFs are printed inside the file of the sum, in a table with 18 columns (first one for `ss`, second for their sum an the next 16 for each effect). 

Otherwise, a new directory "all_standalones_CFs" is created (in the same path given in `out`) and they are separately saved in files there placed.

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(cosmo, 
    "my_first_all_xis.txt", 10 .^ range(0, 3, length = 100);
    L = 0, use_windows = false, alg = :quad);

In [ ]:
ss, res_sums, res_xis = GaPSE.readxyall("my_first_all_xis.txt"; comments = true);

In [ ]:
pyplot()
ps = [plot() for i in 1:length(GaPSE.GR_EFFECTS_GNC) + 1]

title = plot(title = "GNC Correlation functions terms", grid=false, 
    showaxis = false, bottom_margin = -50Plots.px, ticks=false)

legend_pos=[
    :bottomleft,
    :bottomleft, :topright, :bottomleft, :bottomleft,
    :topleft, :topleft, :topleft, :topleft,
    :bottomleft, :bottomleft, :bottomleft, :bottomleft,
    :bottomleft, :bottomleft, :bottomleft, :bottomleft,
]

xs = ss

ps[1] = plot(xs, res_sums .* xs .^2, size=(300, 100),
        lw=2, lc="red", ls=:solid,
        title="All summed xis", 
        xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
        ylabel=L"s^2 \, \xi^{\mathrm{ALL}}_{L=0}(s)",
        #xaxis=:log, #yaxis=:log,
        top_margins=15Plots.px, legend=false)

for i in 1:length(GaPSE.GR_EFFECTS_GNC)
    ys = res_xis[i]

    ps[i+1] = plot(
        xs, ys .* xs .^2, 
        size=(300, 100),
        lw=2, lc="red", ls=:solid,
        title=GaPSE.GR_EFFECTS_GNC[i], 
        xlabel=L"s \quad [h_0^{-1}\,\mathrm{Mpc}]",
        ylabel=L"s^2 \, " * func_ξs_names[i](0),
        #xaxis=:log, #yaxis=:log,
        top_margins=15Plots.px,
        legend = false,
    )
end

l = @layout [A{0.02h}; grid(2,3){0.22h} ; grid(5,4){0.75h} ]
P = plot(title, ps..., layout=l, 
    left_margins = 15Plots.px, right_margins = 30Plots.px,
    size=(1200,1800))

```





```

## Power Spectrum Multipole

Last but not least, from a CF file generated as previously showed, we can easily obtain the corresponding Power Spectrum multipole.
GaPSE provieds two functions:
- `PS_multipole` and `print_PS_multipole`
- `all_PS_multipole` and `print_all_PS_multipole`

```julia
PS_multipole(ss, fs; alg::Symbol=:fftlog,
     cut_first_n::Int=0, cut_last_n::Int=0, kwargs...)
PS_multipole(input::String; comments=true, kwargs...)
```

Return a tuple (`ks`, `pks`) of the Power Spectrum obtained from the CF sampled by (`ss`, `fs`) or stored in the file `input`, through the following expression: 

$$
P_L(k) = \frac{(-i)^L}{A^{'}} \, \phi(s_\mathrm{eff}) \int_0^\infty 
        \mathrm{d} s \; s^2 \, j_L(ks) \, \xi_L(s) \; ,
        \quad \; A^{'} = \frac{1}{4\,\pi}
$$

Currenlty, there are two algorithms you can coose in order to perform the computation; you can choose 
which one to use through the keyword value `alg`:
- `alg = :fftlog` (default and recommended option) will employ the [FFTLog](https://github.com/marcobonici/FFTLog.jl) 
  algorithm.
- `alg = :twofast` will employ the TwoFAST `xicalc` function of the [TwoFAST](https://github.com/hsgg/TwoFAST.jl) 
  Julia package. Note that in the computation the integration range `` 0\\leq s \\leq \\infty``
  is reduced to `int_s_min ≤ s ≤ int_s_max`. This alogrithm is not the ideal choise, because TwoFAST is conceived
  for the direction PS -> TPCF, while is not 100% trustworthy for the other way round.

IMPORTANT: no matter which algorithm you choose, you will need to give the input data in a
LOGARITHMICALLY DISTRIBUTED scale. A linear distribution does not fit for the algorithms to apply.

Depending on the algorithm you choose, the options would change. The options in common are:
- `pr::Bool=true` : want to print the automatic messages to the screen?
- `L::Int=0` : which multipole order should I use for this computation? IT MUST MATCH 
  THE MULTIPOLE ORDER OF THE INPUT TPCF!
- `cut_first_n::Int=0` and `cut_last_n::Int=0` : you can cout the first and/or last n elements
  of the input data, if they are highly irregular.
  
The complete list of options can be obrained thorugh `?PS_multipole`



```julia
print_PS_multipole(input::String, out::String; L::Integer = 0, 
    int_s_min = 1e-3, int_s_max = 1e3, 
    kwargs...) ::Tuple{Vector{Float64}, Vector{Float64}}
```

Same as before, but store the results in a file named `out`.

NOTE: the multipole order `L` must be the same of the input TPCF in exam! Otherwise, the results would have no sense at all!

In [ ]:
GaPSE.print_PS_multipole("my_first_doppler.txt",
    "my_first_ps_doppler.txt"; L = 0)

In [ ]:
xs, ys = GaPSE.readxy("my_first_ps_doppler.txt"; comments = true);
pyplot()
plot(xs, abs.(ys),
    xaxis=:log, yaxis=:log, 
    lc=:red, ls=:solid,size=(500, 300),
    title="auto_doppler PS",
    ylabel=L"P^{v_{\parallel} \, v_{\parallel}}_{L=0}(k)",
    xlabel=L"k \quad [h_0\,\mathrm{Mpc}^{-1}]",
    legend=false,
)

Finally, if we want to operate with a full set of xis, obtained from the group "GNC" for example, we should exploit the final two functions:

```julia
    all_PS_multipole(input::String,
        group::String=VALID_GROUPS[end];
        L::Int=0, pr::Bool=true,
        alg::Symbol=:fftlog, kwargs...)
    print_all_PS_multipole(input::String, out::String,
        group::String=VALID_GROUPS[end];
        L::Int=0, pr::Bool=true,
        alg::Symbol=:fftlog,
        kwargs...)
```

where the argument `group::String=VALID_GROUPS[end]` refers to the name of the group considered. The currently available ones are:

In [ ]:
GaPSE.VALID_GROUPS

where the last one should be used for a generic group of xis, without a specific order.
As an example, we cosider the group "GNC" previously computed:

In [ ]:
function ps_kwargs(alg::Symbol = :fftlog) 
    if alg == :twofast
        return Dict(
            :alg => :twofast, :pr => true,
            :cut_first_n => 6, :cut_last_n => 4,
            :epl => true, 
            :N_left => 12, :N_right => 12,
            :p0_left => [-2.0, 1.0], :p0_right => [-2.0, 1.0],
            :int_s_min => 1e0, :int_s_max => 1200.0,
            cut_first_n => 6, cut_last_n => 3,
            ) 
    elseif alg == :fftlog

        return Dict(
            :alg => :fftlog, 
            :cut_first_n => 6, :cut_last_n => 4,
            :ν => 1.5, :n_pad => 500,
            :n_extrap_low => 0, :n_extrap_high => 0,  
            )
    else
        throw(AssertionError("alg = :$alg is not a valid algorithm for PS_multipole"))
    end
end

 GaPSE.print_all_PS_multipole("my_first_all_xis.txt", "my_first_all_ps.txt", 
    "GNC"; L=0, ps_kwargs(:fftlog)...)

In [ ]:
ks, pks_sums, res_pks = GaPSE.readxyall("my_first_all_ps.txt"; comments = true);

In [ ]:
func_ps_names = [
    L::Int -> L"$P^{\delta \, \delta}_{L=%$L}(s)$", 
    L::Int -> L"$P^{v_{\parallel} \, v_{\parallel}}_{L=%$L}(s)$", 
    L::Int -> L"$P^{\kappa \, \kappa}_{L=%$L}(s)$", 
    L::Int -> L"$P^{\phi \, \phi}_{L=%$L}(s)$", 
    L::Int -> L"$P^{\int\phi \, \int\phi}_{L=%$L}(s)$", 
    
    L::Int -> L"$P^{\delta \, v_{\parallel}}_{L=%$L}(s)$",
    L::Int -> L"$P^{v_{\parallel} \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\delta \, \kappa}_{L=%$L}(s)$",
    L::Int -> L"$P^{\kappa \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\delta \, \phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\phi \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\delta \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\int\phi \, \delta}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\kappa \, v_{\parallel}}_{L=%$L}(s)$",
    L::Int -> L"$P^{v_{\parallel} \, \kappa}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{v_{\parallel} \, \phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\phi \, v_{\parallel}}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{v_{\parallel} \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\int\phi \, v_{\parallel}}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\kappa \, \phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\phi \, \kappa}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\kappa \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\int\phi \, \kappa}_{L=%$L}(s)$",
    
    L::Int -> L"$P^{\phi \, \int\phi}_{L=%$L}(s)$",
    L::Int -> L"$P^{\int\phi \, \phi}_{L=%$L}(s)$",
];

In [ ]:
pyplot()
ps = [plot() for i in 1:length(GaPSE.GR_EFFECTS_GNC) + 1]

title = plot(title = "GNC Correlation functions terms", grid=false, 
    showaxis = false, bottom_margin = -50Plots.px, ticks=false)

legend_pos=[
    :bottomleft,
    :bottomleft, :topright, :bottomleft, :bottomleft,
    :topleft, :topleft, :topleft, :topleft,
    :bottomleft, :bottomleft, :bottomleft, :bottomleft,
    :bottomleft, :bottomleft, :bottomleft, :bottomleft,
]

xs = ks

ps[1] = plot(xs, abs.(pks_sums), size=(300, 100),
        lw=2, lc="red", ls=:solid, axis=:log,
        title="All summed ps", 
        xlabel=L"k \quad [h_0\,\mathrm{Mpc}^{-1}]",
        ylabel=L"P^{\mathrm{ALL}}_{L=0}(s)",
        #xaxis=:log, #yaxis=:log,
        top_margins=15Plots.px, legend=false)

for i in 1:length(GaPSE.GR_EFFECTS_GNC)
    ys = res_pks[i]

    ps[i+1] = plot(
        xs, abs.(ys), 
        size=(300, 100), axis=:log,
        lw=2, lc="red", ls=:solid,
        title=GaPSE.GR_EFFECTS_GNC[i], 
        xlabel=L"k \quad [h_0 \mathrm{Mpc}^{-1}]",
        ylabel=func_ps_names[i](0),
        #xaxis=:log, #yaxis=:log,
        top_margins=15Plots.px,
        legend = false,
    )
end

l = @layout [A{0.02h}; grid(2,3){0.22h} ; grid(5,4){0.75h} ]
P = plot(title, ps..., layout=l, 
    left_margins = 15Plots.px, right_margins = 30Plots.px,
    size=(1200,1800))

## Window Function F and Integrated Window Function 

The bool `use_windows`'s work may be written as follows, with `U = use_windows`:

$$
\xi_L(s_1, s, \mu) = \frac{2 L + 1}{2} \int_{-1}^{+1} \mathrm{d}\mu \; 
    \xi (s_1, s, \mu) \, \mathcal{L}_L(\mu) \,\times
\begin{cases}  
     \mathcal{F(s, \mu)}\;,
        \quad \mathrm{U = true}\\
    1\;, \quad \mathrm{U = false}
\end{cases}
$$

$$
\mathcal{F}(s, \mu) = 
    \int_0^\infty \mathrm{d}s_1 \, \phi(s_1) \,  
    \phi\left(\sqrt{s_1^2 + s^2 + 2 \, s_1 \, s \, \mu}\right) 
    \, F\left(\frac{s}{s_1}, \mu \right)
$$

with $F$ defined as follows (check Eq.(A.10) of Castorina, Di Dio (2021) for more details):

$$
\begin{split}
F(x,\mu; \theta_\mathrm{max}) = & \;4\pi 
    \int_0^{\theta_\mathrm{max}} \mathrm{d}\theta_1 \int_0^\pi \mathrm{d} \theta \; 
    \, \Theta\left(\frac
        {x \cos \theta + \cos \theta_1}{\sqrt{x^1+2+2x\mu}} - 
        \cos(\theta_\mathrm{max}) 
        \right) 
    \, \Theta(\mu-\cos(\theta+\theta_1)) \\
    &\Theta(\cos(\theta - \theta_1)-\mu) \;
    \frac{\sin\theta\sin\theta_1}
        {\sqrt{(\sin\theta\sin\theta_1)^2-(\cos\theta\cos\theta_1-\mu)^2}}
\end{split}
$$

where the $\Theta$s are Heaviside theta functions.

The most efficient way to use $F$ in our computation is to calculate it separetely for a map of $\mu$ and $x$ values (with the $\theta_\mathrm{MAX}$ of the survey), then compute from it $\mathcal{F}$ and load them into the `Cosmology` we want to create. Internally, the code creates a 2d-spline from the input file of $F$.

In order to create suh a file, GaPSE provides the function `F_map`, which has two methods:

```julia
print_F_map(out::String,
    x_step::Float64 = 0.01, μ_step::Float64 = 0.01;
    x1 = 0, x2 = 3, μ1 = -1, μ2 = 1, 
    alg::Symbol = :trap,
    Fmap_opts::Dict = Dict{Symbol,Any}(), 
    kwargs...)

print_F_map(out::String,
    xs::Vector{Float64}, μs::Vector{Float64};
    alg::Symbol = :trap,
    Fmap_opts::Dict = Dict{Symbol,Any}(),
    kwargs...)
```

The first one creates a map of:
- $\mu$ values from `μ1 = -1` to `μ2 = 1` with linear step `μ_step`
- $x$ values from `x1 = 0` to `x2 = 3` with linear step `x_step`
and calculate $F$ in each point of this grid, saving the results in the file `out`.

The symbol keyword argument `alg` tells if you want to perform the computation of `F` with
`F_trap` (if `alg = :trap`) or with `F_hcub` (if `alg = :hcub`). Other symbols will lead to 
an `AssertionError`.
Both for computational efficiency and stability, it's highly recommended to use
the former (i.e. the default one). 

The second one creates a grid from the input vector of $\mu$ and $x$ values, 
calculate $F$ in each point of this grid and save the results in the file `out`.

We now give here an example of use, which corresponds to the code necessary to produce the file `data/F_REFERENCE_pi2.txt`; consider that this computation, for the following values, might takes about 43 minutes. 

In [6]:
#=
reference_output =  PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt"

kwargs_map_F_hcub = Dict(
     :θ_max => π / 2.0, :tolerance => 1e-10, 
     :rtol => 1e-2, :atol => 1e-3, :pr => true,
);

kwargs_map_F_trap = Dict(
     :θ_max => π / 2.0, :tolerance => 1e-10, 
     :N => 1000, :pr => true,
);

xs = [x for x in 0:0.02:5]
μs = union(
    [μ for μ in range(-1.0, -0.98, length = 50)], 
    [μ for μ in range(-0.98, 0.98, length = 102)],
    [μ for μ in range(0.98, 1.0, length = 50)]);
GaPSE.print_map_F(reference_output, xs, μs; 
    alg = :trap, Fmap_opts = kwargs_map_F_trap # we recommend to use :trap
    #alg = :hcub, Fmap_opts = kwargs_map_F_hcub # but you can use also :hcub if you prefer
)
=#

From such a file we can compute the integrated windof function $\mathcal{F}$:

```julia
print_map_IntegratedF(s_min, s_max, ss::Vector{Float64},
     μs::Vector{Float64}, windowF::Union{String,WindowF}, out::String;
     alg::Symbol=:trap, llim=nothing, rlim=nothing,
     rtol=1e-2, atol=0.0, N::Int=1000, pr::Bool=true)
```

You should specify:
- `s_min` and `s_max` for the radial `ϕ` function.
- `ss::Vector{Float64}` and `μs::Vector{Float64}` :  the vector of s and μ points where to evaluate 
  sample the integrated window function ``\\mathcal{F}``; they will be stored in the struct.
- `windowF::Union{String,WindowF}`, i.e. the window function itself; it can be passed as the namefile
  where the window is stored in (that will be opened with `WindowF`) or as a `WindowF` struct directly.
- `out::String` : the name of the output file

As optional arguments:

- `alg::Symbol = :trap` : algorithm to be used for the integration; the valid options are `:quad`
  (that will recall `integrated_F_quadgk`) and `:trap` (that will recall `integrated_F_trapz`);
  other values will lead to `AssertionError`
- `llim=nothing` and `rlim=nothing` : integration limits for quad/trap; 
  if `llim=nothing` it will be set to `0.95 * s_min`; if `rlim=nothing`, it will be set to `1.05*s_max`,
  while if `rlim=Inf` it will be set to `3 * s_max`.
- `N::Int = 1000` : number of points to be used for the sampling of `trapz`; it's useless if you set
  `alg = :quad`;
- `rtol=1e-2` and `atol=0.0` : relative and absoute tolerance for `quadgk`; they are useless if you set
  `alg = :trap`;
- `pr::Bool = true` : do you want to see the progress-bar of the computation?

We now give here an example of use, which corresponds to the code necessary to produce the file `data/IntegrF_REFERENCE_pi2_z005020.txt`; consider that this computation, for the following values, might takes about 3 minutes. 

In [7]:
calc_μs = union(
    [μ for μ in range(-1.0, -0.98, length = 50)], 
    [μ for μ in range(-0.98, 0.98, length = 102)],
    [μ for μ in range(0.98, 1.0, length = 50)]);

GaPSE.print_map_IntegratedF(
    PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt", 
    PATH_TO_GAPSE * "data/IntegrF_REFERENCE_pi2_z005020.txt", 
    calc_μs;
    alg = :trap, Dict(
        :llim => nothing, :rlim => nothing, 
        :rtol => 1e-2, :atol => 0.0, 
        :N => 1000, :pr => true,
    )...
)

LoadError: MethodError: no method matching print_map_IntegratedF(::String, ::String, ::Float64, ::Float64, ::Vector{Float64}; alg=:trap, atol=0.0, rtol=0.01, llim=nothing, N=1000, rlim=nothing, pr=true)
[0mClosest candidates are:
[0m  print_map_IntegratedF(::Any, ::Any, [91m::Vector{Float64}[39m, [91m::Vector{Float64}[39m, [91m::Union{String, Main.GaPSE.WindowF}[39m, [91m::String[39m; alg, llim, rlim, rtol, atol, N, pr) at ~/AAA_TESI_MAGISTRALE/GaPSE/src/WindowFIntegrated.jl:191
[0m  print_map_IntegratedF(::String, [91m::Main.GaPSE.WindowFIntegrated[39m) at ~/AAA_TESI_MAGISTRALE/GaPSE/src/WindowFIntegrated.jl:470[91m got unsupported keyword arguments "alg", "atol", "rtol", "llim", "N", "rlim", "pr"[39m